In [1]:
import pandas as pd
import cerberus
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

In [17]:
species = 'human'
subset = 'all'
gene_subset = 'lncRNA'

In [11]:
def parse_meta(species):
    df = pd.read_csv(f'data/{species}/metadata.tsv', sep='\t')
    df['species'] = species
    df['subset'] = subset
    df['dataset'] = df['tissue']+'_'+\
                 df['age']+'_'+\
                 df['platform']+'_'+\
                 df['capture']+'_'+\
                 df['species']+'_'+\
                 df['subset']
    
    return df

## Look at lncRNA diversity before and after capture

In [ ]:
species = 'human'
obs_col = 'capture'
min_tpm = 1

ca = cerberus.read(f'data/{species}/cerberus/ca_trip.h5')
df = pd.read_csv(f'data/{species}/cerberus/agg_ab.tsv', sep='\t')

# filter on biotype
g_df = pd.read_csv('ref/human/annot_g_info.tsv', sep='\t')
g_df['gid_stable'] = cerberus.get_stable_gid(g_df, 'gid')
g_df = g_df.loc[g_df.biotype_category==gene_subset]
df['gid'] = df.annot_transcript_id.str.split('[', expand=True)[0]
print(len(df.index))
df = df.loc[df.gid.isin(g_df.gid_stable)]
print(len(df.index))
df.drop('gid', axis=1, inplace=True)

df.drop('annot_transcript_name', axis=1, inplace=True)
df.set_index('annot_transcript_id', inplace=True)
df = df.transpose()
df = df.fillna(0)
meta = parse_meta(species)
meta_cols = meta.columns

df = df.merge(meta, how='left', left_index=True, right_on='dataset')
df.drop([c for c in meta_cols if c != obs_col], axis=1, inplace=True)

# groupby and take max tpm value per thing
df = df.groupby('capture').max().reset_index()

# loop through the different conditions
trip = pd.DataFrame()
for obs_cond in df[obs_col].unique():
    print()
    print(f'{obs_col}={obs_cond}')
    source = f'{obs_col}_{obs_cond}'
    temp = df.loc[df[obs_col] == obs_cond].copy(deep=True)
    temp = temp.drop(obs_col, axis=1).transpose()
    temp.columns = ['max_tpm']
    if DEBUG: print(len(temp.index))
    temp = temp.loc[temp.max_tpm >= min_tpm]
    if DEBUG: print(f'{len(temp.index)}\n')
    tids = temp.index.tolist()
    import pdb; pdb.set_trace()
    temp_trip = ca.get_subset_triplets(tids, source)
    _ = plot_feats_per_gene(temp_trip,
                            obs_cond,
                            max_ends=10)
    trip = pd.concat([trip, temp_trip], axis=0)